In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import keras
from keras.layers import Dense, LSTM, SimpleRNN, GRU, Input
from keras.models import Model, Sequential  

df_train = pd.read_csv("sales_train_validation.csv")
#df_sell = pd.read_csv("./storage/Kaggle_M5_Luke/sell_prices.csv")
#df_calendar = pd.read_csv("./storage/Kaggle_M5_Luke/calendar.csv")
#df_ss = pd.read_csv("./storage/Kaggle_M5_Luke/sample_submission_m5.csv")


Using TensorFlow backend.


In [2]:
days = range(1,1913+1) 
time_series_cols = [] 
for i in days: 
    time_series_cols.append('d_'+str(i)) 
time_series_cols[:10]

['d_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9', 'd_10']

In [3]:
time_series_data = df_train[time_series_cols] 
time_series_data

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,0,0,0,0,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0,0,2,2,0,3,1,4,1,0,...,2,0,0,0,0,0,1,0,0,1
30486,0,0,0,0,0,5,0,1,1,3,...,0,0,0,0,0,0,0,0,1,0
30487,0,6,0,2,2,4,1,8,5,2,...,2,1,0,2,0,1,0,0,1,0
30488,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [4]:
time_series_data = np.asarray(time_series_data).astype('float32')
print(time_series_data.shape)

(30490, 1913)


In [12]:
#mean = [] # list of mean values 
#sd = [] # list of standard deviations 

# normalize row by row 
#for i in range(len(time_series_data)):  
#    mean.append(np.mean(time_series_data[i])) 
#    sd.append(np.std(time_series_data[i])) 
#    time_series_data[i] = (time_series_data[i]-np.mean(time_series_data[i]))/np.std(time_series_data[i]) 

In [5]:
time_series_data 

array([[0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 1.],
       ...,
       [0., 6., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 3., 1., 3.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
def build_model(time_steps,features): 
    model = Sequential() 
    model.add(LSTM(128,input_shape=(time_steps,features),activation='relu')) 
    model.add(Dense(30490))   
    return model 

In [7]:
# let's now code for the general case: for all the 28 days 
def split_xy2(dataset, time_steps):
    x,y = list(), list() 
    for i in range(0,dataset.shape[1]-time_steps): 
        x.append(dataset[:,i:i+time_steps]) 
        y.append(dataset[:,i+time_steps]) 
    return np.array(x), np.array(y) 

predictions = [] 

for i in range(1,29): 
    print("day {}".format(i)) 
    dataset = time_series_data 
    x_train, y_train = split_xy2(dataset,7) 
    x_pred = dataset[:,dataset.shape[1]-7:]  
    x_train = x_train.reshape(x_train.shape[0],7,30490) 
    x_pred = x_pred.reshape(1,7,30490)  
    model = build_model(7,30490) 
    model.compile(loss='mse',optimizer='adam',metrics=['mse'])
    model.fit(x_train,y_train,epochs=50) 
    pred = model.predict(x_pred) 
    print(pred[0]) # for checking purposes 
    predictions.append(pred) 
    time_series_data = np.append(time_series_data,pred.T,axis=1)  

#for i in range(1,29): 
#    print("day {}".format(i)) 
#    dataset = time_series_data[:,::i] 
#    x_train, y_train = split_xy2(dataset,7) 
#    x_pred = dataset[:,dataset.shape[1]-7:] 
#    x_train = x_train.reshape(x_train.shape[0],7,30490) 
#    x_pred = x_pred.reshape(1,7,30490) 
#    model = build_model(7,30490) 
#    model.compile(loss='mse',optimizer='adam',metrics=['mse'])
#    model.fit(x_train,y_train,epochs=500) 
#    pred = model.predict(x_pred) 
#    predictions.append(pred) 
    


day 1
Epoch 1/50
1906/1906 [==============================] - 4s 2ms/step - loss: 10.8812 - mse: 10.8812
Epoch 2/50
1906/1906 [==============================] - 2s 1ms/step - loss: 6.6262 - mse: 6.6262
Epoch 3/50
1906/1906 [==============================] - 2s 1ms/step - loss: 6.2576 - mse: 6.2576
Epoch 4/50
1906/1906 [==============================] - 2s 1ms/step - loss: 6.0520 - mse: 6.0520
Epoch 5/50
1906/1906 [==============================] - 2s 1ms/step - loss: 5.8020 - mse: 5.8020
Epoch 6/50
1906/1906 [==============================] - 2s 1ms/step - loss: 5.5717 - mse: 5.5717
Epoch 7/50
1906/1906 [==============================] - 2s 1ms/step - loss: 5.3569 - mse: 5.3569
Epoch 8/50
1906/1906 [==============================] - 2s 1ms/step - loss: 5.2001 - mse: 5.2001
Epoch 9/50
1906/1906 [==============================] - 2s 1ms/step - loss: 5.0879 - mse: 5.0879
Epoch 10/50
1906/1906 [==============================] - 2s 1ms/step - loss: 4.9957 - mse: 4.9957
Epoch 11/50
1906/1906

In [8]:
predictions = np.asarray(predictions) 
print(predictions.shape)

(28, 1, 30490)


In [9]:
predictions

array([[[0.5172829 , 0.0711793 , 0.40716147, ..., 0.8815972 ,
         0.67550534, 1.5009276 ]],

       [[0.4551643 , 0.05685465, 0.41225937, ..., 0.9774723 ,
         0.6996799 , 0.9795997 ]],

       [[0.5779618 , 0.15936662, 0.3922989 , ..., 0.71114385,
         0.6084998 , 1.1446843 ]],

       ...,

       [[0.56945187, 0.2430912 , 0.33058122, ..., 0.54050285,
         0.8393438 , 1.5760018 ]],

       [[0.61750823, 0.49475268, 0.35871515, ..., 0.8676365 ,
         0.8811399 , 1.4517939 ]],

       [[0.54224813, 0.31584746, 0.33590832, ..., 0.5686221 ,
         0.7836888 , 1.6590623 ]]], dtype=float32)

In [10]:
# flatten each of predictions[i] 
pred_flattened = [] 
for i in range(28): 
    pred_flattened.append(predictions[i].flatten()) 

In [11]:
# checking if shape is flattened 
pred_flattened[0]

array([0.5172829 , 0.0711793 , 0.40716147, ..., 0.8815972 , 0.67550534,
       1.5009276 ], dtype=float32)

In [12]:
cols = [] 
for i in range(1,29): 
    cols.append("F"+str(i)) 
df = pd.DataFrame(pred_flattened)   
df = df.T # transpose 
df.columns = cols # rename column 
df 

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,0.517283,0.455164,0.577962,0.528199,0.592449,0.461535,0.743406,0.816472,0.695131,0.417891,...,0.727456,0.834910,0.886344,0.588315,0.716179,0.509193,0.485347,0.569452,0.617508,0.542248
1,0.071179,0.056855,0.159367,0.341209,0.485866,0.368608,0.298897,0.334817,0.360532,0.485550,...,0.336120,0.200912,0.203450,0.220877,0.221934,0.341279,0.199182,0.243091,0.494753,0.315847
2,0.407161,0.412259,0.392299,0.556501,0.598667,0.382207,0.374712,0.360336,0.340894,0.230425,...,0.520199,0.618240,0.275571,0.354207,0.422560,0.369756,0.401865,0.330581,0.358715,0.335908
3,1.515009,1.270727,1.528194,1.498209,1.002943,2.129329,2.218219,1.686663,1.537696,1.540143,...,1.884715,2.065191,2.502917,2.480505,2.436272,1.911247,1.807112,2.023960,2.109646,2.066710
4,1.049078,0.749494,1.196286,1.253804,1.062269,1.141483,1.236588,1.298145,1.327343,0.921687,...,1.236704,0.977390,1.246830,1.031698,0.914970,0.722130,1.050820,1.071879,1.310912,0.787344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,1.000594,0.589039,0.441407,0.546027,0.409646,0.277242,0.596639,0.649822,0.708172,0.667198,...,0.406561,0.407953,0.548208,0.580324,0.733929,0.355859,0.536613,0.548958,0.439558,0.564070
30486,0.223681,0.131635,0.015181,0.062258,0.211503,-0.037544,0.139080,0.056515,0.364780,0.225611,...,0.019802,-0.153591,-0.068329,0.069228,0.003892,0.014568,0.078717,-0.154388,-0.028726,0.102015
30487,0.881597,0.977472,0.711144,0.492233,0.729568,0.741413,0.638344,0.566420,0.496778,0.349326,...,0.819413,0.655746,0.854175,0.803828,0.901303,0.232083,0.431413,0.540503,0.867637,0.568622
30488,0.675505,0.699680,0.608500,1.049339,0.814091,0.543332,1.001550,1.002005,1.012285,0.939085,...,0.874828,0.808648,0.801385,1.067002,0.716950,0.725760,0.619282,0.839344,0.881140,0.783689


In [13]:
validation_ids = df_train['id'].values
evaluation_ids = [i.replace('validation', 'evaluation') for i in validation_ids]
ids = np.concatenate([validation_ids, evaluation_ids])       
ids

array(['HOBBIES_1_001_CA_1_validation', 'HOBBIES_1_002_CA_1_validation',
       'HOBBIES_1_003_CA_1_validation', ...,
       'FOODS_3_825_WI_3_evaluation', 'FOODS_3_826_WI_3_evaluation',
       'FOODS_3_827_WI_3_evaluation'], dtype=object)

In [14]:
sub_df = pd.DataFrame(ids, columns=['id']) 
df = pd.concat([df]*2).reset_index(drop=True) 
sub_df = pd.concat([sub_df,df],axis=1) 
sub_df.to_csv('LSTM_128_append.csv',index=False) 

In [15]:
sub_df 

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.517283,0.455164,0.577962,0.528199,0.592449,0.461535,0.743406,0.816472,0.695131,...,0.727456,0.834910,0.886344,0.588315,0.716179,0.509193,0.485347,0.569452,0.617508,0.542248
1,HOBBIES_1_002_CA_1_validation,0.071179,0.056855,0.159367,0.341209,0.485866,0.368608,0.298897,0.334817,0.360532,...,0.336120,0.200912,0.203450,0.220877,0.221934,0.341279,0.199182,0.243091,0.494753,0.315847
2,HOBBIES_1_003_CA_1_validation,0.407161,0.412259,0.392299,0.556501,0.598667,0.382207,0.374712,0.360336,0.340894,...,0.520199,0.618240,0.275571,0.354207,0.422560,0.369756,0.401865,0.330581,0.358715,0.335908
3,HOBBIES_1_004_CA_1_validation,1.515009,1.270727,1.528194,1.498209,1.002943,2.129329,2.218219,1.686663,1.537696,...,1.884715,2.065191,2.502917,2.480505,2.436272,1.911247,1.807112,2.023960,2.109646,2.066710
4,HOBBIES_1_005_CA_1_validation,1.049078,0.749494,1.196286,1.253804,1.062269,1.141483,1.236588,1.298145,1.327343,...,1.236704,0.977390,1.246830,1.031698,0.914970,0.722130,1.050820,1.071879,1.310912,0.787344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,1.000594,0.589039,0.441407,0.546027,0.409646,0.277242,0.596639,0.649822,0.708172,...,0.406561,0.407953,0.548208,0.580324,0.733929,0.355859,0.536613,0.548958,0.439558,0.564070
60976,FOODS_3_824_WI_3_evaluation,0.223681,0.131635,0.015181,0.062258,0.211503,-0.037544,0.139080,0.056515,0.364780,...,0.019802,-0.153591,-0.068329,0.069228,0.003892,0.014568,0.078717,-0.154388,-0.028726,0.102015
60977,FOODS_3_825_WI_3_evaluation,0.881597,0.977472,0.711144,0.492233,0.729568,0.741413,0.638344,0.566420,0.496778,...,0.819413,0.655746,0.854175,0.803828,0.901303,0.232083,0.431413,0.540503,0.867637,0.568622
60978,FOODS_3_826_WI_3_evaluation,0.675505,0.699680,0.608500,1.049339,0.814091,0.543332,1.001550,1.002005,1.012285,...,0.874828,0.808648,0.801385,1.067002,0.716950,0.725760,0.619282,0.839344,0.881140,0.783689


In [16]:
#model.compile(loss='mse',optimizer='adam',metrics=['mse'])
#model_checkpoint = keras.callbacks.ModelCheckpoint('./storage/Kaggle_M5_Luke/model_base2/model{epoch:02d}.h5',period=50)
#model.fit(x_train,y_train,epochs=1000,callbacks=[model_checkpoint],validation_data=(x_val,y_val))  
